<a href="https://colab.research.google.com/github/moonjune/test-repo/blob/master/khaiii_%ED%85%8C%EC%8A%A4%ED%8A%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!git clone https://github.com/kakao/khaiii.git

In [0]:
!cd khaiii
!mkdir build
!cd build

In [0]:
import os 
os.chdir('/content/khaiii/')

In [0]:
!mkdir build

In [0]:
os.chdir('/content/khaiii/build/')

In [0]:
!cmake ..

In [0]:
!make all

In [0]:
!make resource

In [0]:
os.chdir('/content/khaiii/build/')
!./bin/khaiii --rsc-dir=./share/khaiii

In [0]:
!ctest

In [0]:
!make package_python

In [0]:
os.chdir("/content/khaiii/build/package_python/")
!pip install .

In [0]:
os.chdir("/content/")
!mkdir simmilar_work
os.chdir("/content/simmilar_work/")

In [0]:
os.chdir("/content/simmilar_work/")
import numpy as np
import pandas as pd

In [0]:
# 파일 임포팅 파트
from google.colab import files
uploaded = files.upload()

In [0]:
# 파일 DF로 읽기

RStext = pd.read_excel("191101_테스트셋.xlsx", encoding = "UTF-8")
del RStext["Unnamed: 0"]
RStext.columns

In [0]:
# 통합텍스트 셀 생성
RStext["전체 텍스트"] = RStext['과제명-국문'] + " " + RStext['요약문_연구목표'] + " " + RStext['요약문_연구내용'] + " " + RStext['요약문_기대효과'] + " " + RStext['요약문_한글키워드'] + " " + RStext['요약문_영문키워드']
RStext["전체 텍스트_기본"] = RStext["전체 텍스트"].str.findall('[a-z|A-Z|가-힣|&|.-.]+')
for i in range(len(RStext["전체 텍스트_기본"])):
  RStext["전체 텍스트_기본"][i] = " ".join(RStext["전체 텍스트_기본"][i])

In [0]:
import numpy as np
import pandas as pd
import re
from random import sample
from khaiii import KhaiiiApi
api = KhaiiiApi()

In [0]:
Npercnt = 90

subj_for_test = pd.DataFrame()
subj_for_test["전체 텍스트"] = RStext["전체 텍스트"].str.split(" ")
subj_for_test["제목 길이"] = 0
subj_for_test["목표 길이"] = 0
subj_for_test["내용 길이"] = 0
subj_for_test["효과 길이"] = 0
subj_for_test["한키 길이"] = 0
subj_for_test["영키 길이"] = 0
subj_for_test["제목 추출"] = ''
subj_for_test["목표 추출"] = ''
subj_for_test["내용 추출"] = ''
subj_for_test["효과 추출"] = ''
subj_for_test["한키 추출"] = ''
subj_for_test["영키 추출"] = ''

In [0]:
type(subj_for_test["전체 텍스트"][0])

In [0]:
for i in range(len(RStext)):
  sub_text = re.sub("\t|\n|\r","",str(" " + " ".join(subj_for_test["전체 텍스트"][i])+" "))
  subj_for_test["제목 길이"][i] = len(RStext["과제명-국문"][i]) + 1
  subj_for_test["목표 길이"][i] = int(subj_for_test["제목 길이"][i]) + len(RStext["요약문_연구목표"][i])
  subj_for_test["내용 길이"][i] = int(subj_for_test["목표 길이"][i]) + len(RStext["요약문_연구내용"][i])
  subj_for_test["효과 길이"][i] = int(subj_for_test["내용 길이"][i]) + len(RStext["요약문_기대효과"][i])
  subj_for_test["한키 길이"][i] = int(subj_for_test["효과 길이"][i]) + len(RStext["요약문_한글키워드"][i])
  subj_for_test["영키 길이"][i] = int(subj_for_test["한키 길이"][i]) + len(RStext["요약문_영문키워드"][i])
  test = []
  for word in api.analyze(RStext["전체 텍스트"][i]):
    test.append(str(word))
    test_str = re.sub('[A-Za-z가-힣]*\t'," "," ".join(test))
    test_str = re.compile('[A-Za-z가-힣]*/NNG|[A-Za-z가-힣]*/SL|[A-Za-z가-힣]*/NNP').findall(test_str)
    test_str = re.sub('/NNG|/SL|/NNP',""," ".join(test_str))
    test_str = re.compile('[A-Za-z가-힣][A-Za-z가-힣]+').findall(test_str)
    test_set = list(set(test_str))
    test_str = str(" "+" ".join(test_str)+" ")
    subj_count = pd.DataFrame(np.zeros([len(test_set),2]),columns = ["키워드","빈도"])
    subj_count["위치"] = ""
    subj_count["위치변화"] = ""
    subj_count["변경수"] = 0
    subj_count["변경대상"] = ""
    for k in range(len(test_set)):
      subj_count["키워드"][k] = test_set[k]
      subj_count["빈도"][k] = len(re.compile(str(" "+test_set[k]+" ")).findall(test_str))
      subj_count["위치"][k] = [m.start() for m in re.finditer(subj_count["키워드"][k],sub_text)]
      subj_count["위치변화"][k] = sample(subj_count["위치"][k],len(subj_count["위치"][k]))
    subj_count = subj_count.sort_values(by = ["빈도"],ascending=False).reset_index(drop=True)
    Keycount = int(round(sum(subj_count["빈도"])/100*(100-Npercnt),0))
    for j in range(len(subj_count)):
      subj_count["변경수"][j] = int(np.random.randint(0, high = int(subj_count["빈도"][j])+1,size = 1))*int(sample([0,1,1,1],1)[0])
      if Keycount < 0:
        subj_count["변경수"][j] = subj_count["변경수"][j] - (0 - Keycount)
        subj_count["변경대상"][j] = list(subj_count["위치변화"][j][0:subj_count["변경수"][j]])
        break
      subj_count["변경대상"][j] = list(subj_count["위치변화"][j][0:subj_count["변경수"][j]])
  for l in range(len(subj_count)):
    if len(subj_count["변경대상"][l]) == 0:
      continue
    else:
      for ll in range(len(subj_count["변경대상"][l])):
        sub_text = "".join((sub_text[:subj_count["변경대상"][l][ll]],"연구",sub_text[subj_count["변경대상"][l][ll]+len(subj_count["키워드"][l]):]))
  subj_for_test["제목 추출"][i] = sub_text[0:subj_for_test["제목 길이"][i]]
  subj_for_test["목표 추출"][i] = sub_text[subj_for_test["제목 길이"][i]:subj_for_test["목표 길이"][i]]
  subj_for_test["내용 추출"][i] = sub_text[subj_for_test["목표 길이"][i]:subj_for_test["내용 길이"][i]]
  subj_for_test["효과 추출"][i] = sub_text[subj_for_test["내용 길이"][i]:subj_for_test["효과 길이"][i]]
  subj_for_test["한키 추출"][i] = sub_text[subj_for_test["효과 길이"][i]:subj_for_test["한키 길이"][i]]
  subj_for_test["영키 추출"][i] = sub_text[subj_for_test["한키 길이"][i]:subj_for_test["영키 길이"][i]]



In [0]:
subj_for_test["영키 길이"][i]

In [0]:
subj_for_test.head(10)

In [0]:
sub_text
[m.start() for m in re.finditer(subj_count["키워드"][k],sub_text)]

In [0]:
# # 
# test_str = re.sub('[A-Za-z가-힣]*\t'," "," ".join(test))
# test_str = re.compile('[A-Za-z가-힣]*/NNG|[A-Za-z가-힣]*/SL|[A-Za-z가-힣]*/NNP').findall(test_str)
# test_str = re.sub('/NNG|/SL|/NNP',""," ".join(test_str))
# test_str = re.compile('[A-Za-z가-힣][A-Za-z가-힣]+').findall(test_str)
# test_set = list(set(test_str))
# test_str = str(" "+" ".join(test_str)+" ")
# print(test_str)
# print(test_set)

In [0]:
# subj_count = pd.DataFrame(np.zeros([len(test_set),2]),columns = ["키워드","빈도"])
# subj_count["위치"] = ""
# subj_count["위치변화"] = ""
# subj_count["변경수"] = 0
# subj_count["변경대상"] = ""
# for i in range(len(test_set)):
#   subj_count["키워드"][i] = test_set[i]
#   subj_count["빈도"][i] = len(re.compile(str(" "+test_set[i]+" ")).findall(test_str))
#   subj_count["위치"][i] = list(np.where(np.array(test_str.split(" ")) == test_set[i])[0])
#   subj_count["위치변화"][i] = sample(subj_count["위치"][i],len(subj_count["위치"][i]))
# subj_count = subj_count.sort_values(by = ["빈도"],ascending=False).reset_index(drop=True)
# Npercnt = 10
# Keycount = int(round(sum(subj_count["빈도"])/100*Npercnt,0))
# for j in range(len(subj_count)):
#   subj_count["변경수"][j] = int(np.random.randint(0, high = int(subj_count["빈도"][j])+1,size = 1))*int(sample([0,1,1,1],1)[0])
#   Keycount -= subj_count["변경수"][j]
#   if Keycount < 0:
#     subj_count["변경수"][j] = subj_count["변경수"][j] - (0 - Keycount)
#     subj_count["변경대상"][j] = list(subj_count["위치변화"][j][0:subj_count["변경수"][j]])
#     break
#   subj_count["변경대상"][j] = list(subj_count["위치변화"][j][0:subj_count["변경수"][j]])


In [0]:
subj_count.sort_values(by =["빈도"],ascending=False)[0:30]

In [0]:
print(sub_text[382])
print(subj_count["키워드"][l])
print(subj_count["변경대상"][l][ll])
print(l)
print(ll)

In [0]:
print(int(round(sum(subj_count["빈도"])/100*Npercnt,0)))
print(sum(subj_count["변경수"]))

In [0]:
print(int(round(sum(subj_count["빈도"])/10*2,0)))

In [0]:
print(RStext["과제고유번호"][num])
print(RStext["과제명-국문"][num])

In [0]:
# 단어 체인지 후에 각 필드로 배분하기 위한 방법
print(len(RStext["전체 텍스트"][num].split(" ")))
print(len(RStext["과제명-국문"][num].split(" ")))
print(RStext["전체 텍스트"][num].split(" ")[0:len(RStext["과제명-국문"][num].split(" "))])
print(RStext["과제명-국문"][num].split(" "))
print(RStext["전체 텍스트"][num].split(" ")[len(RStext["과제명-국문"][num].split(" ")):(len(RStext["과제명-국문"][num].split(" "))+len(RStext["요약문_연구목표"][num].split(" ")))])
print(RStext["요약문_연구목표"][num].split(" "))
print(RStext["전체 텍스트"][num].split(" ")[len(RStext["과제명-국문"][num].split(" ")):(len(RStext["과제명-국문"][num].split(" "))+len(RStext["요약문_연구목표"][num].split(" ")))])

In [0]:
print(RStext["요약문_연구내용"][num].split(" "))

In [0]:
np.where(np.array(RStext["요약문_연구내용"][num].split(" ")) == "분석")[0]